In [ ]:
#VENTOY
#https://ostechnix.com/how-to-create-multiboot-usb-drives-with-ventoy-in-linux/
#F1 load to ram
wget https://github.com/ventoy/Ventoy/releases/download/v1.0.17/ventoy-1.0.17-linux.tar.gz
tar -xvjf ventoy-1.0.17-linux.tar.gz
cd ventoy*
sudo fdisk -l
#sudo sh Ventoy2Disk.sh -I /dev/sdc
sudo sh Ventoy2Disk.sh -I -s /dev/sdc
#Alternatively, go to the location where you saved ISO files and copy all ISO files from command line with rsync like below:
#rsync *.iso /media/$USER/ventoy/ --progress -ah

choco install -y ventoy
sudo snap install ventoy
brew cask install ventoy
mount ESP
copy modified esp partition
copy ventoy json


In [ ]:
#Refind
#Format: 
sudo umount /dev/sdd1
sudo mkfs.vfat -F 32 /dev/sdd1
#Mount: 
sudo mkdir /tmp/usbboot
sudo mount /dev/sdd1 /tmp/usbboot
#Prepare destination folder: 
sudo mkdir /tmp/usbboot/EFI
#Copy rEFInd's files: 
#cp -a /path/to/refind /tmp/usbboot/EFI/ (or use the refind-install script: 
sudo refind-install --root /tmp/usbboot
#Edit rEFInd's configuration to taste (
#sudo nano /tmp/usbboot/efi/refind.conf


#EFI
#https://stackoverflow.com/questions/7878707/how-to-unmount-a-busy-device
#https://howtoubuntu.org/how-to-repair-restore-reinstall-grub-2-with-a-ubuntu-live-cd
#https://support.system76.com/articles/bootloader/
#https://www.tonymacx86.com/threads/help-booting-systemd-arch-efi-with-clover.188191/
sudo mount /dev/nvme1n1p3 /mnt
sudo mount /dev/nvme1n1p1 /mnt/boot/efi
for i in /dev /dev/pts /proc /sys /run; do sudo mount -B $i /mnt$i; done
sudo cp /etc/resolv.conf /mnt/etc/
sudo chroot /mnt
apt install --reinstall grub-efi-amd64 linux-generic linux-headers-generic
update-initramfs -c -k all
exit
sudo update-grub
sudo umount /mnt/boot/efi
sudo umount -l /mnt

#SYSTEMD
sudo rm -rf /boot/efi/*
sudo mount /dev/nvme1n1p3 /mnt
sudo mount /dev/nvme1n1p1 /mnt/boot/efi
for i in /dev /dev/pts /proc /sys /run; do sudo mount -B $i /mnt$i; done
sudo cp /etc/resolv.conf /mnt/etc/
sudo chroot /mnt
apt install --reinstall linux-generic linux-headers-generic
update-initramfs -c -k all
exit
sudo bootctl --path=/mnt/boot/efi install
sudo umount /mnt/boot/efi
sudo umount -l /mnt


#qcow22mount
#https://www.jamescoyle.net/how-to/1818-access-a-qcow2-virtual-disk-image-from-the-host
sudo modprobe nbd
sudo qemu-nbd -c /dev/nbd0 ~/Documents/ESP.qcow2
sudo partx -l /dev/nbd0
sudo mkdir /mnt/ESP
sudo mount /dev/nbd0p1 /mnt/ESP
#sudo cp -rf /mnt/ESP/* /media/user/SED/VBOX/ESP/

#CLOVER INSTALL
git clone https://github.com/m13253/clover-linux-installer.git
cd clover*
./install.sh
#/dev/nvme1n1
#/dev/nvme1n1p1
#/dev/sdc
#/dev/sdc1

#mountESP
#W10
disk="nvme0n1p1"

#MacOS
disk="sdc1"
#PopOS
disk="nvme1n1p1"
loc="/home/$USER/Documents"
folder="ESP0"
full="$loc/$folder"
sudo umount $full
sudo mkdir $full
sudo mount /dev/$disk $full
sudo nautilus $full
sudo umount $full

#MacOS
disk="sdb1"
sudo umount ~/Documents/ESPC
sudo mkdir ~/Documents/ESPC
sudo mount /dev/$disk ~/Documents/ESPC
sudo nautilus ~/Documents/ESPC


#copy apfs.efi to ESP
#cd /media/user/SED/VBOX/ESP/EFI/CLOVER
#sudo cp -r ./drivers64UEFI /mnt/MOEFI/EFI/CLOVER/drivers64UEFI
sudo rm -rf /mnt/ESP0/*
sudo cp -rf /media/user/SED/VBOX/ESP/* /mnt/ESP0/
sudo umount /mnt/ESP0/

#/boot/efi/EFI/CLOVER
#sudo ls /boot/efi/EFI/CLOVER
#sudo rm -rf /boot/efi/EFI/CLOVER
sudo cp -rf /media/user/SED/VBOX/ESP/* /boot/efi/

In [ ]:
sudo apt install -y autoconf libaio-dev libbluetooth-dev libbrlapi-dev libbz2-dev libcap-dev libcap-ng-dev libcurl4-gnutls-dev libepoxy-dev libfdt-dev libgbm-dev libgles2-mesa-dev libglib2.0-dev libgtk-3-dev libibverbs-dev libjpeg8-dev liblzo2-dev libncurses5-dev libnuma-dev librbd-dev librdmacm-dev libsasl2-dev libsdl1.2-dev libsdl2-dev libseccomp-dev libsnappy-dev libssh2-1-dev libspice-server-dev libspice-server1 libtool libusb-1.0-0 libusb-1.0-0-dev libvde-dev libvdeplug-dev libvte-dev libxen-dev valgrind xfslibs-dev xutils-dev zlib1g-dev libusbredirhost-dev usbredirserver

#virglrenderer
git clone git://git.freedesktop.org/git/virglrenderer
cd virglrenderer
./autogen.sh
make -j7
sudo make install

#https://unix.stackexchange.com/questions/268460/how-to-mount-qcow2-image
#Step 1 - Enable NBD on the host
sudo modprobe nbd max_part=8
#Step 2 - Connect the QCOW2 as a network block device
sudo qemu-nbd --connect=/dev/nbd0 /var/lib/libvirt/images/ChromeOS.qcow2
#Step 3 - List partitions inside the QCOW2
sudo fdisk /dev/nbd0 -l
#Step 4 - Mount the partition from the VM
#sudo mount /dev/nbd0p1 /mnt/somepoint/


#COS
sudo apt-get update
sudo apt-get install -y figlet 
sudo apt-get install -y pv
sudo apt-get install -y cgpt
cd /media/user/Ventoy/ChromeOS
sudo bash chromeos-install.sh -src rammus_recovery.bin -dst /dev/nbd0

#Step 5 - After you're done, unmount and disconnect
#sudo umount /mnt/somepoint/
sudo qemu-nbd --disconnect /dev/nbd0
sudo rmmod nbd


cd /media/user/Ventoy/ChromeOS
/usr/bin/qemu-system-x86_64 \
    -enable-kvm \
    -m 2G \
    -smp 4 \
    -hda /media/user/Ventoy/ChromeOS/rammus_recovery.bin \
    -vga virtio \
    -net nic,model=virtio \
    -net user,hostfwd=tcp:127.0.0.1:9222-:22 \
    -usb -usbdevice keyboard \
    -usbdevice mouse \
    -device virtio-gpu-pci,virgl \
    -display gtk,gl=on